In [6]:
import dask.dataframe as dd
import dask.bag as db
docs = dd.read_parquet("output.pq/", columns=['News_Id', 'News_Title', 'News_Text'])

In [2]:
docs['News_Query'] = docs.apply(lambda row: [row['News_Title'], row['News_Text']], axis=1, meta=('News_Query', 'object'))

In [3]:
import pyarrow as pa
schema = {
        "News_Id": pa.int32(),
        "News_Text": pa.string(),
        "News_Title": pa.string(),
        "News_Query": pa.list_(pa.string())
    }
docs.to_parquet("query.pq", schema = schema)

# Добавление запросов из LLM

In [1]:
from transformers import pipeline

chatbot = pipeline("text-generation", model="microsoft/Phi-4-mini-instruct", max_new_tokens=256)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


In [10]:
import re

def parse_queries(llm_q: str) -> list[str]:
    # Разбиваем по строкам
    lines = llm_q.strip().split('\n')
    queries = []
    for line in lines:
        # Убираем номер и точку в начале строки, например "1. "
        query = re.sub(r'^\s*\d+\.\s*', '', line)
        queries.append(query.strip())
    return queries

promt_template = lambda news: f"""
        Вот текст новости:\n{news}"""

def generate_query(text: str) -> list[str]:
    messages = [
        {"role": "system", "content": """Тебе дают текст новости. 
         Сгенерируй 5 реалистичных поисковых запросов, которые могли бы ввести пользователи в поисковике, чтобы найти эту новость. 
         Учитывай, что пользователи часто не знают точных формулировок, могут использовать ключевые слова, имена, места, приблизительные описания и синонимы. 
         Запросы должны быть короткими (3-7 слов). Формулируй запросы разнообразно и естественно, как это делают реальные пользователи."""},
        {"role": "user", "content": promt_template(text)}
    ]
    llm_q = chatbot(messages)[0]['generated_text'][2]['content']
    return parse_queries(llm_q)

In [ ]:
sample_df = docs.sample(frac=0.0001, random_state=42)
# sample_df = docs.loc[0].compute()

In [ ]:
sample_df['News_Query'] = sample_df['News_Text'].map(generate_query,  meta=("News_Tokens", object)).compute()
print(sample_df.head())


TokenizationError: Object <function <lambda> at 0x0000017BC79C51C0> cannot be deterministically hashed. This likely indicates that the object cannot be serialized deterministically.

In [26]:
print(sample_df['News_Query'])

Dask Series Structure:
npartitions=12
    string
       ...
     ...  
       ...
       ...
Dask Name: getitem, 5 expressions
Expr=(AssignAlign(frame=Sample(frame=ReadParquetFSSpec(99bda07), state_data=[array([1608637542, 3421126067, 4083286876,  787846414, 3143890026,
       3348747335, 2571218620, 2563451924,  670094950, 1914837113,
        669991378,  429389014,  249467210, 1972458954, 3720198231,
       1433267572, 2581769315,  613608295, 3041148567, 2795544706,
         88409749,  242285876, 4165731073, 3100961111, 3575313899,
       4031053213,  911989541,    3344769,  780932287, 4261516219,
        787716372, 2652062880, 1306710475, 2627030329, 2253811733,
         30349564, 1855189739,   99052376, 1250819632, 2253890010,
       2627888186, 1717389822,  599121577,  200427519, 1254751707,
       4182248123, 1573512143,  999745294, 1958805693,  389151677,
       3372305070, 2655947709,  857592370, 1642661739, 2208620086,
       4222944499, 2544401215, 2004731384,  199502978, 3693

In [29]:
import pyarrow as pa
schema = {
        "News_Id": pa.int64(),
        "News_Text": pa.string(),
        "News_Title": pa.string(),
        "News_Query": pa.string()
    }
sample_df.to_csv('llm-query.pq')

['c:\\Users\\infer\\Desktop\\Курсовя\\Thesaurus-construction\\llm-query.pq\\0.part',
 'c:\\Users\\infer\\Desktop\\Курсовя\\Thesaurus-construction\\llm-query.pq\\1.part',
 'c:\\Users\\infer\\Desktop\\Курсовя\\Thesaurus-construction\\llm-query.pq\\2.part',
 'c:\\Users\\infer\\Desktop\\Курсовя\\Thesaurus-construction\\llm-query.pq\\3.part']